In [1]:
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf
import keras

2025-06-18 10:10:04.197066: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 10:10:04.240153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-18 10:10:04.240177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-18 10:10:04.241008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-18 10:10:04.247263: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# human antigens files
df_rep = pd.DataFrame(glob('/projects/deeptcr/DeepTCR/Data/Human_Antigens/*/*.tsv'), columns=['filepath'])
df_rep[['HLA', 'Antigen']] = df_rep['filepath'].str.extract(r'/([^/-]+)-([^/-]+)/[^/]+$')

In [3]:
# load tcrseq data and label
df_tcr = []
for idx in df_rep.index:
    df_tcr.append(pd.read_csv(df_rep.loc[idx, 'filepath'], sep='\t'))
    df_tcr[-1]['index'] = idx
    df_tcr[-1][['HLA', 'Antigen']] = df_rep.loc[idx, ['HLA', 'Antigen']].values
df_tcr = pd.concat(df_tcr)
df_tcr['Antigen'] = df_tcr['Antigen'].astype('category')

In [4]:
# convert to tf dataset
idx = ~df_tcr[['aminoAcid', 'Antigen']].isna().any(axis=1)
X = df_tcr.loc[idx, 'aminoAcid'].values
y = df_tcr.loc[idx, 'Antigen'].cat.codes.values


In [5]:
max_length = max(list(map(len, X)))
aa = set(''.join(X))
aa = dict(zip(aa, np.arange(1, len(aa) + 1)))

X = [[aa[l] for l in x] + [0] * (max_length - len(x)) for x in X]



In [6]:
tfds = tf.data.Dataset.from_tensor_slices((X, y))
tfds = tfds.map(lambda X, y_true: (X, tf.one_hot(y_true, len(np.unique(y)))))
tfds = tfds.shuffle(idx.sum(), reshuffle_each_iteration=True)
tfds = tfds.batch(100, drop_remainder=True)

2025-06-18 10:10:14.714407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14778 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:d8:00.0, compute capability: 7.5


In [7]:
tfds

<_BatchDataset element_spec=(TensorSpec(shape=(100, 23), dtype=tf.int32, name=None), TensorSpec(shape=(100, 7), dtype=tf.float32, name=None))>

In [8]:
# CDR3 AA encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [ ]:
# VDJ usage encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [9]:
tensors = []
tensors.append(keras.layers.Input(shape=(encoder.output_shape[-1], ), dtype=tf.float32))
tensors.append(keras.layers.Dropout(rate=0.1)(tensors[-1]))
tensors.append(keras.layers.Dense(units=128, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dropout(rate=0.05)(tensors[-1]))
tensors.append(keras.layers.Dense(units=64, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dense(units=len(np.unique(y)), activation=keras.activations.sigmoid)(tensors[-1]))
classifier = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='classifer')

In [10]:
tensors = []
tensors.append(keras.layers.Input(shape=encoder.input_shape[1:],  dtype=tf.uint32))
tensors.append(encoder(tensors[-1]))
tensors.append(classifier(tensors[-1]))
model = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='full_model')

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(from_logits=False)
)

In [13]:
model.fit(tfds, epochs=100)

Epoch 1/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3560
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3501
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3452
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3356
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3253
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3154
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3044
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2903
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2724
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2544
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2336
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2060
Epoch 13/100
20/20 [================

In [151]:
y_pred.shape

(2000, 7)

In [157]:
b = next(iter(tfds))

In [158]:
b[0].shape, b[1].shape

(TensorShape([100, 23]), TensorShape([100, 7]))